In [1]:
import pandas as pd
# import datetime
# from dateutil.relativedelta import relativedelta
import FinanceDataReader as fdr
# import numpy as np
# import re
# from marcap import marcap_data
import time
# import matplotlib.pyplot as plt
# from pykrx import stock
from datetime import datetime

# 가격 데이터 저장

In [2]:
df_kospi = fdr.StockListing('KOSPI') # KOSPI, KOSDAQ
df_kosdaq = fdr.StockListing('KOSDAQ')
kospi_list = df_kospi['Symbol'].to_list()
kosdaq_list = df_kosdaq['Symbol'].to_list()
krx_del_list = fdr.StockListing('KRX-DELISTING')['Symbol'].to_list()

## 종가 자료

In [4]:
df_base = pd.DataFrame()
start=time.time()
krx_list = kospi_list+kosdaq_list+krx_del_list
error_list=[]
for code in krx_list:
    try:
        df_temp = fdr.DataReader(code).loc[:,'Close'].to_frame()
        df_temp.rename(columns = {'Close' : code}, inplace=True)
        df_base = pd.concat([df_base,df_temp],axis=1)
    except:
        error_list.append(code)
print(time.time()-start)
print(len(error_list))
# df_base
df_base.to_csv('Raw_Price/price_all.csv', index=True, encoding='utf-8-sig')

2917.737733602524
5641


## 변화율 자료

In [3]:
df_base = pd.DataFrame()
start=time.time()
krx_list = kospi_list+kosdaq_list+krx_del_list
error_list=[]
for code in krx_list:
    try:
        df_temp = fdr.DataReader(code).loc[:,'Change'].to_frame()
        df_temp.rename(columns = {'Change' : code}, inplace=True)
        df_base = pd.concat([df_base,df_temp],axis=1)
    except:
        error_list.append(code)
print(time.time()-start)
print(len(error_list))
# df_base
df_base.to_csv('Raw_Price/change_all.csv', index=True, encoding='utf-8-sig')

2866.6382863521576
6387


## 인덱스 자료

In [8]:
df_change = fdr.DataReader('KS11')
df_change.to_csv('Raw_Price/kospi_index.csv', index=True, encoding='utf-8-sig')

## 종가 자료 불러오기

In [ ]:
# csv 읽어서 dataframe 다시 만들기
price = pd.read_csv('Raw/Raw/price_all.csv',low_memory=False)
# index 지정
price = price.set_index(price.columns[0])
# .1 삭제
del_list=[]
for i in price.columns:
    if '.1' in i:
        del_list.append(i)
price.drop(columns = del_list, inplace=True)
price

## 변화율 자료 불러오기

In [3]:
## 종가 자료 불러오기

# csv 읽어서 dataframe 다시 만들기
price = pd.read_csv('Raw/Raw/price_all.csv',low_memory=False)
# index 지정
price = price.set_index(price.columns[0])
# .1 삭제
del_list=[]
for i in price.columns:
    if '.1' in i:
        del_list.append(i)
price.drop(columns = del_list, inplace=True)
price# csv 읽어서 dataframe 다시 만들기
change = pd.read_csv('Raw/Raw/change_all.csv',low_memory=False)
# index 지정
change = change.set_index(change.columns[0])
# .1 삭제
del_list=[]
for i in change.columns:
    if '.1' in i:
        del_list.append(i)
change.drop(columns = del_list, inplace=True)
change = change.fillna(-1) # 값이 없는 경우 상장폐지로 보고, -1을 넣어서 수익률에 고려하게 적용

## 시가총액 data 불러오기

In [4]:
from pykrx import stock
import pandas as pd
df1 = stock.get_market_ohlcv_by_ticker("20210618", "KOSPI")
df2 = stock.get_market_ohlcv_by_ticker("20210618", "KOSDAQ")
df = pd.concat([df1,df2],axis=0)

In [6]:
df1 = stock.get_market_cap_by_ticker("20210618", "KOSPI")
df2 = stock.get_market_cap_by_ticker("20210618", "KOSDAQ")
df = pd.concat([df1,df2],axis=0)
df

,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
005930,80500,480567495275000,14916721,1202369230300,5969782550
000660,124500,90636294442500,2506112,314308082000,728002365
035720,155000,68809085675000,7706345,1193066108000,443929585
035420,398000,65376831210000,1268656,505640525610,164263395
005935,74000,60893615800000,1426525,105832676600,822886700
...,...,...,...,...,...
114570,620,7030424280,0,0,11339394
335870,2245,7004400000,240479,539102500,3120000
080440,498,6636328080,0,0,13325960


## 거래대금 소액 제외 및 우선주 제외하고 시가총액 순으로 정렬

In [9]:
import numpy as np
import re
cost_list = df.sort_values(by='시가총액',ascending=True)
cost_list = cost_list.loc[lambda cost_list:cost_list['거래대금']> np.int64('100000000'),:] #거래대금 1억 이상
total_list = cost_list.index
list_temp=[]
p=re.compile('[0-9]*5$|[0-9]*7$|[0-9]*9$|[0-9]*K$|[0-9]*L$|[0-9]*M$|[0-9]*B$')
for i in total_list:
    if not p.match(i):
        list_temp.append(i)
cost_list = cost_list.loc[list_temp,:] # 우선주 제외
# cost_list = cost_list.index
cost_list

,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
335870,2245,7004400000,240479,539102500,3120000
347140,2350,7071150000,730050,1702558920,3009000
323210,2275,7280000000,2894177,6652117825,3200000
311270,2145,7722000000,3539578,7824419735,3600000
373200,2215,7796800000,479147,1072850950,3520000
...,...,...,...,...,...
051910,822000,58026905946000,252978,209227194000,70592343
035420,398000,65376831210000,1268656,505640525610,164263395
035720,155000,68809085675000,7706345,1193066108000,443929585


In [ ]:
def date_str (date):
    date_temp=str("%s-%02d-%02d" %(date.year, date.month, date.day))
    return date_temp
def date_str1 (date):
    date_temp=str("%s%02d%02d" %(date.year, date.month, date.day))
    return date_temp
# 1개월 이전의 기간 동안의 수익률

# def first_date (date):
    

def Change_acc (corp_code, date, duration):
    gap=int('1') # 매수매도 판단 시점 기준으로 몇달 전의 수익을 가지고 판단하는지
    start = date_str(date-relativedelta(months=(int(duration)+gap)))
    end = date_str(date-relativedelta(months=gap,days=1))
    
    df_stock_yield=fdr.DataReader(corp_code,start,end)
    df_stock_yield['Change_acc']=(1+df_stock_yield['Change']).cumprod()
    return df_stock_yield.tail(1)['Change_acc'][0]

def output_dart (text):
    txt = 'Raw/Dart/{}.txt'.format(text)
    dart = pd.read_csv(txt, encoding = 'CP949', sep = '\t')
    dart1 = dart.loc[:,['종목코드','회사명','결산월','결산기준일','보고서종류','통화','항목명','당기']]

    word_list = []
    for i in dart1['항목명'].index:
        word = dart1['항목명'][i]
        word = re.compile('[^|가-힣]+').sub('',str(word))
        word_list.append(word)
    dart1 = dart1.drop(['항목명'],axis=1)
    dart1.rename(columns = {'당기':'2019 손익계산서'},inplace=True)
    dart1.insert(loc=(len(dart1.columns)-1),column='항목명',value=word_list)
    return dart1

# dataframe 정렬하기
def dataframe_reset(df,stock_code):
    # 첫열을 index로 수정하기
    df.index = df.iloc[:,0]
    df = df.drop(columns = df.columns[0])
    df.index.name = None
    
    # 결측 columns 삭제하기
    data_temp = df.isnull().sum().to_frame()
    nan_list = data_temp.index[data_temp[0] == len(df.index)]
    df = df.drop(columns=nan_list)
        
    # column에 사용할 하나의 tuple type행으로 만들기
    list_temp=[]
    df_columns_temp=[]
    for x in range(len(df.columns)):
        for y in range(len(df.index)):
            list_temp.append((df.columns[x][0],df.columns[x][1],df.index[y]))
    tuples = list_temp        
    index = pd.MultiIndex.from_tuples(tuples)
    # 빈 dataframe만들기
    df_temp = pd.DataFrame(index=[stock_code], columns=index)
        
    # 빈 dataframe에 값 입력하기
    # Unnamed: 이면 패스하기 / Unnamed 가 여러개 라서 error가 발생하는 듯.
    for j in range(len(df.columns)):
        for i in range(len(df.index)):
            df_temp.at[stock_code,(df.columns[j][0],df.columns[j][1],df.index[i])]=df.loc[df.index[i],(df.columns[j][0],df.columns[j][1])]
    dataframe_stock = df_temp
    return dataframe_stock

def yield_fdr(stock_list,bgn,end):
    dist_list = []
    for j in range(len(stock_list)):
        dist_list.append(1/len(stock_list))

    date1 = date_str(bgn)
    date2 = date_str(end)

    # 수익률 dataframe으로 만드는 함수 추가
    df_yield = pd.DataFrame()
    for i in range(len(stock_list)):
        df_temp = fdr.DataReader(stock_list[i],date1,date2)['Change'].to_frame()
        df_temp['Change_acc']=(1+df_temp['Change']).cumprod() * dist_list[i]
        df_temp = df_temp.drop(['Change'],axis=1)
        df_temp = df_temp.rename(columns={'Change_acc':stock_list[i]})
        df_yield = pd.concat([df_yield,df_temp],axis=1)
    df_yield.loc[date1:date2,'Change_all']=df_yield.sum(axis=1)
    return df_yield

def yield_change(stock_list,bgn,end): # 현재 사용 중
    # 수익률 자료에 없는 종목들이 걸려서 제거.
    stock_list_temp = []
    for i in stock_list:
        if str('%06d'%(int(i))) in change.columns:
            stock_list_temp.append(str('%06d'%(int(i))))
        else:
            print(str('%06d'%(int(i))))

    stock_list = stock_list_temp
    
    dist_list = []
    for j in range(len(stock_list)):
        dist_list.append(1/len(stock_list))

    date1 = date_str(bgn)
    date2 = date_str(end)

    # 수익률 dataframe으로 만드는 함수 추가
    df_yield = pd.DataFrame()
    for i in range(len(stock_list)):
        stock_code = '%06d'%(int(stock_list[i])) # 앞에 0이 사라져 다시 변환함
        df_temp = change.loc[date1:date2][stock_code].to_frame()
        df_temp.columns = ['Change']
        df_temp['Change_acc']=(1+df_temp['Change']).cumprod() * dist_list[i]
        df_temp = df_temp.drop(['Change'],axis=1)
        df_temp = df_temp.rename(columns={'Change_acc':stock_code})
        df_yield = pd.concat([df_yield,df_temp],axis=1)
    df_yield.loc[date1:date2,'Change_all']=df_yield.sum(axis=1)
    return df_yield

def price_change(list,bgn,end):
    date1 = date_str(bgn)
    date2 = date_str(end)
    df = price.loc[date1:date2, list]
    df_change = pd.DataFrame(columns = df.columns, index = df.index)
    for j in range(len(df.columns)):
        for i in range(len(df.index)-1):
            df_change.iloc[0,j] = 0
            df_change.iloc[i+1,j] = df.iloc[i+1,j] / df.iloc[i,j]-1
        df_change.loc[:,df.columns[j]] = (1+df_change[df.columns[j]]).cumprod()/len(df.columns)
    df_change.loc[:,'Change_all']=df_change.sum(axis=1)
    return df_change

def yield_fdr_simple(stock_list,bgn,end):
    dist_list = []
    for j in range(len(stock_list)):
        dist_list.append(1/len(stock_list))

    date1 = date_str(bgn)
    date2 = date_str(end)
    
    # 수익률 dataframe으로 만드는 함수 추가
    df_yield = pd.DataFrame()

    for i in stock_list:
        price1 = fdr.DataReader(i,date1,date2).iloc[0,3]
        price2 = fdr.DataReader(i,date3,date4).iloc[0,3]
        df_temp = pd.DataFrame(data=[price1,price2,price2/price1*dist_list[j]],columns=[i],index=[date1,date3,'yield'])
        df_yield = pd.concat([df_yield,df_temp],axis=1)
    df_yield.loc[:,'Change_all']=df_yield.sum(axis=1)
    return df_yield

def making_cost_list (bgn):
    date1 = date_str(bgn)
    date2 = date_str(bgn+relativedelta(days=7))
    first_date=fdr.DataReader('005930',date1,date2).index[0]
    df1 = stock.get_market_ohlcv_by_ticker(date_str1(first_date), "KOSPI")
    df2 = stock.get_market_ohlcv_by_ticker(date_str1(first_date), "KOSDAQ")
    df = pd.concat([df1,df2],axis=0)
    cost_list = df.sort_values(by='시가총액',ascending=True)
    cost_list = cost_list.loc[lambda cost_list:cost_list['거래대금']> np.int64('100000000'),:] #거래대금 1억 이상
    total_list = cost_list.index
    list_temp=[]
    p=re.compile('[0-9]*5$|[0-9]*7$|[0-9]*9$|[0-9]*K$|[0-9]*L$|[0-9]*M$|[0-9]*B$')
    for i in total_list:
        if not p.match(i):
            list_temp.append(i)
    cost_list = cost_list.loc[list_temp,:] # 우선주 제외
    cost_list = cost_list.index
    return cost_list